In [1]:
# Setup library paths
import os
import numpy as np
import SimpleITK as sitk

from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as pdf
%matplotlib inline
plt.rcParams['figure.figsize'] = [30, 15]

In [2]:
def volumetric_dice(img, mask):
    img[img > 1] = 1
    img[img < 1] = 0
    mask[mask > 1] = 1
    mask[mask < 1] = 0
    return np.sum(mask[img == mask]) * 2.0 / (np.sum(img)+np.sum(mask))

In [3]:
root_path = "/Users/amithkamath/repo/deep-planner"
base_gt_path = os.path.join(root_path, "data", "processed-ONL")
base_pred_path = os.path.join(root_path, "results", "output-ONL", "Prediction")

In [5]:
predicted_ONL_dose = []

for pred_idx in range(0, 10):
    pred_dose = sitk.ReadImage(os.path.join(base_pred_path, "DLDP_" + str(pred_idx).zfill(3), "Dose.nii.gz"))
    pred_dose = sitk.GetArrayFromImage(pred_dose)
    
    pred_mask = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(pred_idx).zfill(3), "OpticNerve_L.nii.gz"))
    pred_mask = sitk.GetArrayFromImage(pred_mask)

    mean_pred = np.mean(pred_dose[pred_mask > 0])
    predicted_ONL_dose.append(mean_pred)

print(predicted_ONL_dose)

[35.54512, 35.767376, 32.374123, 34.51261, 34.062115, 30.108437, 35.963806, 41.17088, 33.05762, 36.06449]


In [ ]:
expert_conf_matrix = np.zeros((10, 10))
dice_conf_matrix = np.zeros((10, 10))

for first_idx in range(0, 10):
    for second_idx in range(0, 10):
        first_dose = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(first_idx).zfill(3), "Dose.nii.gz"))
        first_dose = sitk.GetArrayFromImage(first_dose)
        
        second_dose = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(second_idx).zfill(3), "Dose.nii.gz"))
        second_dose = sitk.GetArrayFromImage(second_dose)
        
        first_mask = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(first_idx).zfill(3), "OpticNerve_L.nii.gz"))
        first_mask = sitk.GetArrayFromImage(first_mask)

        second_mask = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(second_idx).zfill(3), "OpticNerve_L.nii.gz")) 
        second_mask = sitk.GetArrayFromImage(second_mask)

        mean_first = np.mean(first_dose[first_mask > 0])
        mean_second = np.mean(second_dose[second_mask > 0])
        diff = np.abs(mean_first - mean_second)
        expert_conf_matrix[first_idx - 1, second_idx - 1] = diff
        dice_conf_matrix[first_idx - 1, second_idx - 1] = volumetric_dice(first_mask, second_mask)

print(np.sum(np.abs(expert_conf_matrix), axis=1))

[21.9929987  24.61257634 21.15027571 29.61777099 43.91977061 21.44198947
 80.15611953 25.86707505 22.90430834 21.15027571]


In [ ]:
model_conf_matrix = np.zeros((10, 10))

for first_idx in range(0, 10):
    for second_idx in range(0, 10):
        first_dose = sitk.ReadImage(os.path.join(base_pred_path, "DLDP_" + str(first_idx).zfill(3), "Dose.nii.gz"))
        first_dose = sitk.GetArrayFromImage(first_dose)
        
        second_dose = sitk.ReadImage(os.path.join(base_pred_path, "DLDP_" + str(second_idx).zfill(3), "Dose.nii.gz"))
        second_dose = sitk.GetArrayFromImage(second_dose)
        
        first_mask = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(first_idx).zfill(3), "OpticNerve_L.nii.gz"))
        first_mask = sitk.GetArrayFromImage(first_mask)

        second_mask = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(second_idx).zfill(3), "OpticNerve_L.nii.gz")) 
        second_mask = sitk.GetArrayFromImage(second_mask)

        mean_first = np.mean(first_dose[first_mask > 0])
        mean_second = np.mean(second_dose[second_mask > 0])
        diff = np.abs(mean_first - mean_second)
        model_conf_matrix[first_idx - 1, second_idx - 1] = diff

print(np.sum(np.abs(model_conf_matrix), axis=1))

[20.84127808 29.41672516 20.39676666 21.29776001 47.54221344 21.6269989
 63.08221436 25.31573486 22.23110962 20.39676666]


In [ ]:
expert_conf_matrix

array([[ 0.        ,  2.37273706,  0.64158674,  3.31147772,  5.09922767,
         0.13775231,  7.30836134,  2.68636173,  0.15188494,  0.28360919],
       [ 2.37273706,  0.        ,  1.73115032,  0.93874067,  2.72649062,
         2.23498475,  9.68109839,  0.31362468,  2.524622  ,  2.08912787],
       [ 0.64158674,  1.73115032,  0.        ,  2.66989098,  4.45764093,
         0.50383443,  7.94994808,  2.04477499,  0.79347168,  0.35797755],
       [ 3.31147772,  0.93874067,  2.66989098,  0.        ,  1.78774995,
         3.17372542, 10.61983906,  0.62511599,  3.46336266,  3.02786854],
       [ 5.09922767,  2.72649062,  4.45764093,  1.78774995,  0.        ,
         4.96147537, 12.40758901,  2.41286594,  5.25111262,  4.81561849],
       [ 0.13775231,  2.23498475,  0.50383443,  3.17372542,  4.96147537,
         0.        ,  7.44611365,  2.54860943,  0.28963725,  0.14585688],
       [ 7.30836134,  9.68109839,  7.94994808, 10.61983906, 12.40758901,
         7.44611365,  0.        ,  9.99472307

In [ ]:
model_conf_matrix

array([[ 0.        ,  3.39325333,  1.25476456,  1.70526123,  5.65893936,
         0.19643021,  5.40350342,  2.70975494,  0.29711533,  0.22225571],
       [ 3.39325333,  0.        ,  2.13848877,  1.6879921 ,  2.26568604,
         3.58968353,  8.79675674,  0.68349838,  3.69036865,  3.17099762],
       [ 1.25476456,  2.13848877,  0.        ,  0.45049667,  4.4041748 ,
         1.45119476,  6.65826797,  1.45499039,  1.55187988,  1.03250885],
       [ 1.70526123,  1.6879921 ,  0.45049667,  0.        ,  3.95367813,
         1.90169144,  7.10876465,  1.00449371,  2.00237656,  1.48300552],
       [ 5.65893936,  2.26568604,  4.4041748 ,  3.95367813,  0.        ,
         5.85536957, 11.06244278,  2.94918442,  5.95605469,  5.43668365],
       [ 0.19643021,  3.58968353,  1.45119476,  1.90169144,  5.85536957,
         0.        ,  5.20707321,  2.90618515,  0.10068512,  0.41868591],
       [ 5.40350342,  8.79675674,  6.65826797,  7.10876465, 11.06244278,
         5.20707321,  0.        ,  8.11325836

The idea here is we compare the pairwise difference of the mean dose scores between the 0th contour - which we think is the 'actual' one, and 9 other alternatives. When we compare the dose scores between the expert dose plans, we see that the 0th plan is closest to the 0th plan - which is expected, and then the 5th plan is the next closest, then the 8th, finally 4 and 6.

Using now the predicted plans, the order of closeness is maintained reasonably well - where the 5th plan is also the closest, and the 6th and 4th plans are the farthest. 

In [ ]:
expert_order = np.argsort(np.abs(expert_conf_matrix[:, 0]))
pred_order   = np.argsort(np.abs(model_conf_matrix[:, 0]))
dice_order   = np.argsort(dice_conf_matrix[:, 0])[::-1]

print("Similarity list of experts    : ", expert_order)
print("Similarity list of predictions: ", pred_order)
print("Similarity list of by dice    : ", dice_order)

print("Dose differences among experts    : ", np.abs(expert_conf_matrix[:, 0])[expert_order])
print("Dose differences among predictions: ", np.abs(model_conf_matrix[:, 0])[expert_order])
print("Dice score differences            : ", np.abs(dice_conf_matrix[:, 0])[expert_order])

Similarity list of experts    :  [0 5 8 9 2 1 7 3 4 6]
Similarity list of predictions:  [0 5 9 8 2 3 7 1 6 4]
Similarity list of by dice    :  [0 9 2 4 1 5 8 7 3 6]
Dose differences among experts    :  [0.         0.13775231 0.15188494 0.28360919 0.64158674 2.37273706
 2.68636173 3.31147772 5.09922767 7.30836134]
Dose differences among predictions:  [0.         0.19643021 0.29711533 0.22225571 1.25476456 3.39325333
 2.70975494 1.70526123 5.65893936 5.40350342]
Dice score differences            :  [1.         0.30769231 0.26373626 0.62790698 0.58928571 0.50877193
 0.20408163 0.16438356 0.58252427 0.04878049]
